In [5]:
# Step 2: Import libraries
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
import urllib.request
import tkinter as tk
from tkinter import filedialog
import os


In [7]:
# Step 3: Load ResNet-101 Model
model = models.resnet101(pretrained=True)
model.eval()


C:\Users\91910\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\91910\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\91910/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:54<00:00, 3.25MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# Step 4: Download ImageNet class labels
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels = urllib.request.urlopen(LABELS_URL).read().decode().splitlines()


In [11]:
# Step 5: Define preprocessing transform
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])


In [13]:
# Step 6: Image loader and predictor
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
    return image, input_batch

def predict(input_batch):
    with torch.no_grad():
        output = model(input_batch)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    return probabilities


In [15]:
# Step 7: Classify and show result
def classify_image(image_path):
    image, input_batch = load_image(image_path)
    probs = predict(input_batch)
    top_prob, top_catid = torch.topk(probs, 1)
    label = labels[top_catid]

    plt.imshow(image)
    plt.title(f"Prediction: {label} ({top_prob.item()*100:.2f}%)")
    plt.axis("off")
    plt.show()


In [17]:
# Step 8: Use file dialog to select image
def get_image_from_user():
    root = tk.Tk()
    root.withdraw()  # Hide tkinter root window
    file_path = filedialog.askopenfilename(
        title="Select a Dog or Cat Image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    return file_path


In [23]:
# Step 9: Run the full classifier
img_path = get_image_from_user()

if img_path:
    print(f"📸 Selected: {img_path}")
    classify_image(img_path)
else:
    print("⚠️ No image selected.")


⚠️ No image selected.
